In [1]:
import requests
import re
import pandas as pd
import numpy as np
import sys
from bs4 import BeautifulSoup
import time
import datetime

In [2]:
# Центр Москвы
c = np.array([37.609218, 55.753559])
# Различные округи Москвы.
# Как оказалось, не все из них действительно существуют,
# так что все равно придется делать дополнительную проверку позже
district = np.hstack((1,np.arange(4,133,1)))

In [4]:
### ==================
### Общие функции:

In [5]:
# Сколько страниц для данного округа
# Считаем на основе: 25 квартир на странице
# (топ 3, который высвечивается на каждой странице, мы благополучно всегда пропускаем)
def page_counter(count):
    r = count/25
    if ((float(count)/25 - count/25)>0): r=r+1
    return r

In [6]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [7]:
### Функции для каждой отдельной страницы с квартирой:

In [8]:
# Номер квартиры берем прямо из ссылки на нее.
def get_N(n):
    N = str(''.join(filter(lambda x: x.isdigit(), n)))
    return N

In [9]:
# Номер этажа, берется как первое число из пары "Этаж: N / M",
# где N - номер этажа, M - всего этажей.
def get_floor(page):
    i = str(page).find('Этаж:</th>')
    if(i==-1): return ''
    p1 = str(page)[i:i+60]
    p1 = html_stripper(p1)
    i = str(p1).find('\n\n')
    p1 = str(p1)[i+4:i+30]
    p1 = re.sub(' ', '', str(p1))
    p1 = re.sub('\n', '', str(p1))
    flo = p1.split('\xc2')[0]
    return flo

In [10]:
# Количество этажей в доме
# Обычно запись идет в виде N / M,
# где N - номер этажа, M - всего этажей.
# Иногда число M просто отсутствует, это учтено..
def get_Nfloor(page):
    i = str(page).find('Этаж:</th>')
    if(i==-1): return ''
    p1 = str(page)[i:i+60]
    p1 = html_stripper(p1)
    i = str(p1).find('\n\n')
    p1 = str(p1)[i+4:i+30]
    p1 = re.sub(' ', '', str(p1))
    p1 = re.sub('\n', '', str(p1))
    i = str(p1).find('\xc2\xa0/\xc2\xa0')
    if(i==-1): return '' # ..здесь
    flo = p1.split('\xc2\xa0/\xc2\xa0')[1]
    return flo

In [11]:
# Количество комнат ищем по инварианту "N-комн. кв."
# N - это однозначное число
# Если этого инварианта нет, в виде исключения возвращается знак ">"
def get_rooms(page):
    i = str(page).find('-комн. кв.')
    p1 = str(page)[i-1:i]
    if(p1=='>'): return ''
    return str(p1)

In [12]:
# Ищем слово "новостройка" для определения первичного рынка
s = 'новостройка'
def get_new(page):
    if(str(page).find(s)==-1): return '0'
    else: return '1'

In [13]:
# Все просто, ищем наличие одного из следующих слов и выдаем 1 или 0
# Если не нашли таких слов на странице, то вернем 0
# Обращаю внимание, что если слово найдено в описании к квартире, то оно также учтется (вроде логично)
t1 = 'панельный' # жб
t2 = 'монолитно-кирпичный'
t3 = 'монолитный'
t4 = 'блочный' # жб
t5 = 'кирпичный'
t6 = 'деревянный'
t7 = 'сталинский'
def get_type(page):
    if(str(page).find(t1)==-1):
        if(str(page).find(t2)==-1):
            if(str(page).find(t3)==-1):
                if(str(page).find(t4)==-1):
                    if(str(page).find(t5)==-1):
                        if(str(page).find(t6)==-1):
                            if(str(page).find(t7)==-1):
                                return '0'
                            else: return '0' # t7
                        else: return '0' # t6
                    else: return '1' # t5
                else: return '1' # t4
            else: return '1' #t3
        else: return '1' # t2
    else: return '1' # t1

In [14]:
# Определяем цену
def get_price(page):
    temp = html_stripper(page.find('div', attrs= {'class':'object_descr_price'}))
    temp = re.sub(' ', '', str(temp))
    temp = re.sub('\n', '', str(temp))
    temp = temp.split('\xd1')[0]
    i = temp.find('$~') # -1 иникатор того, что цена указана не в рублях
    if(i==-1):
        return str(temp)
    else: # если все же цена указана в долларах
        temp = temp.split('$~')[1] # оставляем только рубли
        return str(temp)

In [15]:
# Ищем площадь по ключевому слову "Общая площадь"
def get_sq(page):
    i = str(page).find('Общая площадь:</th>')
    if(i==-1): return ''
    temp = str(page)[i+79:i+100]
    temp = temp.split('\xc2')[0]
    temp = re.sub(',', '.', (temp))
    return str(temp)

In [16]:
# Ищем ключевые слова "Жилая площадь" и "Площадь кухни"
# Если их нет, вернем пустую строку
def get_live_sq(page):
    i = str(page).find('Жилая площадь:</th>')
    if(i==-1): return ''
    temp = str(page)[i+112:i+120]
    temp = temp.split('\xc2')[0]
    temp = re.sub(',', '.', (temp)) # разделитель дробной части меняем на точку
    temp = re.sub('–\n', '', str(temp)) # обработа пустого значения
    # '–\n': данный набор символов появится в следующем случае: "Жилая площадь: –"
    return str(temp)
def get_kit_sq(page):
    i = str(page).find('Площадь кухни:</th>')
    if(i==-1): return ''
    temp = str(page)[i+112:i+120]
    temp = temp.split('\xc2')[0]
    temp = re.sub(',', '.', (temp))
    temp = re.sub('–\n', '', str(temp))
    return str(temp)

In [17]:
# Если на странице есть упоминание о расстоянии до метро, то его возвращаем
# Если упоминания о метро нет, то возвращаем пустую строку
def get_metro_dist(page):
    i = str(page).find('objects_item_metro_prg')
    if(i==-1): return ''
    ans = np.array([0, 0])
    temp = html_stripper(page.find('span', attrs= {'class':'object_item_metro_comment'}))
    temp = re.sub(' ', '', str(temp))
    temp = re.sub('\n', '', str(temp))
    temp = temp.split('\xd0')[0]
    return str(temp)

In [18]:
# Если на странице есть упоминание о расстоянии до метро, то ищем ключевое слово "пешком" и возвращаем 1, иначе 0
# Если упоминания о метро нет, то возвращаем пустую строку
def get_metro_walk(page):
    i = str(page).find('objects_item_metro_prg')
    if(i==-1): return ''
    temp = html_stripper(page.find('span', attrs= {'class':'object_item_metro_comment'}))
    temp = re.sub(' ', '', str(temp))
    temp = re.sub('\n', '', str(temp))
    i = str(temp).find('пешком')
    if(i==-1): return '0'
    else: return '1'

In [19]:
# Ставим 1 только если на странице есть слово "телефон" и после него идет "да"
def get_tel(page):
    temp = str(page).find('Телефон:</th>')
    if(temp==-1): return '0'
    temp1 = str(page)[temp+20:temp+35]
    temp = temp1.find('да')
    if(temp==-1): return '0'
    else: return '1'

In [20]:
# Ищем слово "Балкон", после него либо будет указано N балк., либо M лодж., либо и то, и то.
# Если нашли балк. или лодж. - ставим единицу
def get_balk(page):
    temp = str(page).find('Балкон:</th>')
    if(temp==-1): return '0'
    temp1 = str(page)[temp:temp+400]
    temp = temp1.find('балк.')
    if(temp>-1): return '1'
    temp = temp1.find('лодж.')
    if(temp==-1): return '0'
    else: return '1'   

In [21]:
# Проверяем наличие ссылки на яндекс карты
# Опытным путем убеждаемся, что координаты записаны так: pt=y,x
# Причем именно в данном порядке
def get_dist(page):
    i = str(page).find('yandex.ru')
    if(i==-1): return ''
    
    temp = str(page)[i:i+100]
    i = str(temp).find('pt=')
    if(i==-1): return ''
    
    temp = temp.split('pt=')[1]
    i = str(temp).find(',')
    if(i==-1): return ''
    
    y = float(temp.split(',')[0])
    x = float(temp.split(',')[1])
    return ((x-c[0])**2+(y-c[1])**2)**(0.5)

In [22]:
### ==================
### Сам процесс:

In [23]:
url_1 = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D='
url_2 = '&engine_version=2&offer_type=flat'
url_page = '&p='
url_3 = '&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [ ]:
data = pd.DataFrame(columns=['N', 'Rooms', 'Price', 'Totsp', 'Livesp', 'Kitsp', 'Dist', 'Metrdist', 'Walk', 'Brick', 'Tel', 'Bal', 'Floor', 'Nfloors', 'New'])
start_time = datetime.datetime.now() # Засекаем общее время
for ar in district: # Цикл по округам (их порядка 120)
    start_time_district = datetime.datetime.now() # Засекаем время для округа
    url = url_1 + str(ar) + url_2 + url_3
    page = BeautifulSoup(requests.get(url).content, 'lxml')
    co = html_stripper(page.find('strong'))
    if(co=='None'): # Проверка на существование округа
        print 'District ', ar, ' does not exist'
        continue
    count = int(co)
    p = page_counter(count)
    if(p>30): p = 31 # 30 - это максимум, который видно по одному фиксированному запросу
    else: p = p + 1
    i=1
    while(i<p): # Цикл по страницам (их обычно ровно 30)
        if(i>1):
            url = url_1 + str(ar) + url_2 + url_page + str(i) + url_3
            page = BeautifulSoup(requests.get(url).content, 'lxml')
        
        del_top3 = 0
        help_page = pd.DataFrame(columns=['link'])
        for link in page.findAll('a'): # Ищем все ссылки на квартиры и записываем из в help_page
            tmp=link.get('href')
            if(tmp[:28]=='http://www.cian.ru/sale/flat'): # Оставляем ссылки только на квартиры
                del_top3 = del_top3 + 1
                if(del_top3<4):
                    continue # Топ 3 нам не нужен (он на каждой странице одинаковый, все равно все 3 рано или поздно появятся)
                add = {'link':tmp}
                help_page = help_page.append(add, ignore_index=True)
        
        for link in help_page['link']: # Цикл по квартирам на одной странице (их обычно ровно 25)
            N = get_N(link[29:]) # Номер квартиры берем прямо из ссылки
            page = BeautifulSoup(requests.get(link).content, 'lxml') # Страница с квартирой
            # Для каждого параметра используется отдельная функция
            # Сделана попытка учесть все исключения и вернуть пустое значение в их случае.
            floor = get_floor(page)
            Nfloor = get_Nfloor(page)
            brick = get_type(page)
            rooms = get_rooms(page)
            new = get_new(page)
            price = get_price(page)
            sq = get_sq(page)
            live = get_live_sq(page)
            kit = get_kit_sq(page)
            metro_d = get_metro_dist(page)
            metro_w = get_metro_walk(page)            
            tel = get_tel(page)
            bal = get_balk(page)
            dist = get_dist(page)
            
            # Записываем полученные данные о квартире
            add = {'N': N, 'Rooms': rooms, 'Price': price, 'Floor': floor, 'Totsp': sq, 'Livesp': live, 'Kitsp': kit,'Dist': dist, 'Metrdist': metro_d, 'Walk': metro_w, 'Nfloors': Nfloor, 'Brick': brick, 'Tel': tel, 'Bal': bal, 'New': new}
            data = data.append(add, ignore_index=True)
        i=i+1
    print 'District:', ar, ', time:', datetime.datetime.now() - start_time_district
print '==='
print 'All time:', datetime.datetime.now() - start_time

District: 1 , time: 0:15:47.261000
District: 4 , time: 0:16:27.396000
District: 5 , time: 0:15:52.172000


In [610]:
data

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New
0,148995377,1,3579593,23.8,,,0.266382,15,1,1,0,0,7,24,1
1,150331343,1,3900000,15,9,2,0.190881,3,1,1,1,0,4,5,1
2,148995972,1,3945639,36.1,,,0.266382,15,1,1,0,0,2,23,1
3,148996031,1,3953287,36.2,,,0.266382,15,1,1,0,0,2,23,1
4,148996153,1,4063819,36.2,,,0.266382,15,1,1,0,0,2,18,1
5,148995362,1,4259225,32.6,,,0.266382,15,1,1,0,0,3,24,1
6,148995811,1,4378530,37.4,,,0.266382,15,1,1,0,0,3,25,1
7,148995687,1,4446410,37.9,,,0.266382,15,1,1,0,0,21,25,1
8,148995702,1,4528394,37.5,,,0.266382,15,1,1,0,0,4,25,1
9,148995733,1,4571072,37.9,,,0.266382,15,1,1,0,0,20,25,1


In [611]:
data.to_csv('C:\\_CMF\\ParsingWebData_CIAN_Rakhamatulin_Yan.csv')

In [ ]:
### Больше тут ничего нет!